In [1]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
def xml_to_csv(path):
  xml_list = []
  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      value = (
               root.find('filename').text,
               int(root.find('size/width').text),
               int(root.find('size/height').text),
               member.find('name').text,
               int(member.find('bndbox/xmin').text),
               int(member.find('bndbox/ymin').text),
               int(member.find('bndbox/xmax').text),
               int(member.find('bndbox/ymax').text)
               )
      xml_list.append(value)
      print(f'Added value: {value}')
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  print(f'Total entries: {len(xml_list)}')
  xml_df = pd.DataFrame(xml_list, columns=column_name)
  return xml_df

In [21]:
for folder in ['train', 'test']:
  image_path = f'C:/Users/jackson.feldman/Desktop/cbdata/images/{folder}'
  xml_df = xml_to_csv(image_path)
  xml_df.to_csv(f'C:/Users/jackson.feldman/Desktop/cbdata/annotations/{folder}_labels.csv', index=None)
  print(f'converted to csv for {folder}')

Added value: ('1_ul.png', 816, 902, 'checked', 56, 287, 71, 301)
Added value: ('cb2.png', 770, 677, 'checked', 48, 166, 71, 199)
Added value: ('cb2.png', 770, 677, 'checked', 559, 145, 579, 167)
Added value: ('cb2.png', 770, 677, 'checked', 71, 438, 94, 469)
Added value: ('cb2.png', 770, 677, 'checked', 558, 202, 576, 220)
Added value: ('Screenshot 2024-07-17 094738.png', 875, 828, 'checked', 68, 273, 101, 336)
Added value: ('Screenshot 2024-07-17 094738.png', 875, 828, 'checked', 50, 371, 76, 433)
Added value: ('Screenshot 2024-07-17 094738.png', 875, 828, 'checked', 625, 358, 652, 386)
Added value: ('Screenshot 2024-07-17 094738.png', 875, 828, 'checked', 632, 402, 652, 426)
Added value: ('Screenshot 2024-07-17 094738.png', 875, 828, 'checked', 46, 461, 69, 515)
Added value: ('Screenshot 2024-07-17 094738.png', 875, 828, 'checked', 621, 453, 652, 478)
Added value: ('Screenshot 2024-07-17 094738.png', 875, 828, 'checked', 628, 496, 651, 520)
Added value: ('Screenshot 2024-07-17 094738

In [2]:
train_df = pd.read_csv('C:/Users/jackson.feldman/Desktop/cbdata/annotations/train_labels.csv')
train_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,1_ul.png,816,902,checked,56,287,71,301
1,cb2.png,770,677,checked,48,166,71,199
2,cb2.png,770,677,checked,559,145,579,167
3,cb2.png,770,677,checked,71,438,94,469
4,cb2.png,770,677,checked,558,202,576,220


In [2]:
%pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
def class_text_to_int(row_label):
    if row_label == "checked":
        return 1
    else:
        return None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [18]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_image = fid.read()
    encoded_image_io = io.BytesIO(encoded_image)
    image = Image.open(encoded_image_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'png' if group.filename.endswith('.png') else b'jpeg'

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [20]:
for folder in ['train', 'test']:
    writer = tf.io.TFRecordWriter(f'{folder}.record')
    path = f'C:/Users/jackson.feldman/Desktop/cbdata/images/{folder}'
    examples = pd.read_csv(f'C:/Users/jackson.feldman/Desktop/cbdata/annotations/{folder}_labels.csv')
    grouped = split(examples, 'filename')

    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    
    writer.close()
    print(f'created tfrecord for {folder}.record')

created tfrecord for train.record
created tfrecord for test.record


In [8]:
# Adjust path to your Object Detection API installation
object_detection_path = "C:/Users/jackson.feldman/models/research/object_detection"
os.chdir(object_detection_path)

# Command to start training
!python model_main_tf2.py --model_dir="C:/Users/jackson.feldman/Desktop/training" --pipeline_config_path="C:/Users/jackson.feldman/ssd_mobilenet_v1_coco_11_06_2017/pipeline.config"


2024-07-19 12:51:31.213443: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-19 12:51:32.043588: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "C:\Users\jackson.feldman\models\research\object_detection\model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
  File "C:\Users\jackson.feldman\AppData\Roaming\Python\Python312\site-packages\object_detection\model_lib_v2.py", line 29, in <module>
    from object_detection import eval_util
  File "C:\Users\jackson.feldman\AppData\Roam